# Brewed Insights: Coffee Sales Analysis

### 1. Install and import the relevant libraries

In [1]:
import sqlite3
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import numpy as np

### 2. Load data

In [2]:
# Load CSV data into a Pandas DataFrame
base_path = os.path.dirname(os.getcwd())  # go up from notebooks folder
data_path = os.path.join(base_path, 'data', 'coffee_sales.csv')

df = pd.read_csv(data_path)

### 3. Handle Cross-Year Data (Mar 2024 - Mar 2025)

In [ ]:
# Parse the Date and Time columns
df['Date'] = pd.to_datetime(df['Date'])
# Handle inconsistent time formats (some have milliseconds, some don't)
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), format='mixed') # mixed format parsing

# Extract Year for calculations
df['Year'] = df['Date'].dt.year

# Create sequential month number (handles cross-year properly)
# This counts unique year-month combinations
df['month_sequence'] = (
    (df['Date'].dt.year - df['Date'].dt.year.min()) * 12 + 
    df['Date'].dt.month
)
# Normalize to start from 1
df['month_sequence'] = df['month_sequence'] - df['month_sequence'].min() + 1

# Create display labels for months (e.g., "Mar 2024", "Apr 2024")
df['year_month_display'] = df['Date'].dt.strftime('%b %Y')

# Create year_month_sort for proper ordering
df['year_month_sort'] = df['Date'].dt.year * 100 + df['Date'].dt.month

# Create fiscal year fields (March = Month 1)
FISCAL_START_MONTH = 3  # March
df['fiscal_month'] = ((df['Date'].dt.month - FISCAL_START_MONTH) % 12) + 1 # Fiscal month 1 = March
df['fiscal_year'] = df['Date'].dt.year # Initial fiscal year
df.loc[df['Date'].dt.month < FISCAL_START_MONTH, 'fiscal_year'] -= 1 # Adjust fiscal year for Jan/Feb

# Create fiscal period label for display
df['fiscal_period'] = 'FY' + df['fiscal_year'].astype(str) + '-M' + df['fiscal_month'].astype(str).str.zfill(2)

print("✓ Cross-year data handling complete")
print(f"Data spans: {df['year_month_display'].min()} to {df['year_month_display'].max()}")
print(f"Total unique months: {df['month_sequence'].nunique()}")
print(f"Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")

✓ Cross-year data handling complete
Data spans: Apr 2024 to Sep 2024
Total unique months: 13
Date range: 2024-03-01 to 2025-03-23


### 4. Create a temporary database using SQLite and insert the table

In [9]:
# Create a temporary SQLite database
conn = sqlite3.connect('coffee.db')

# Write the DataFrame into a SQL table
df.to_sql('coffee_sales', conn, index=False, if_exists='replace')

3547

### 5. Create a function that will be reutilized later

In [10]:
def run_query(query, conn):
    """Helper function to run SQL queries and return DataFrame."""
    return pd.read_sql_query(query, conn)

### 6. Exploratory Data Analysis (EDA)

#### 6.1 Top-Selling Coffee Products

The top 5 best-selling drinks (ordered by total revenue) are:

1. **Latte** – 757 units sold, $26,875.30 revenue  
2. **Americano with Milk** – 809 units sold, $24,751.12 revenue  
3. **Capuccino** – 486 units sold, $17,439.14 revenue  
4. **Americano** – 564 units sold, $14,650.26 revenue  
5. **Cortado** – 287 units sold, $7,384.86 revenue

**Strategy Recommendations:**
- **Menu focus**: Promote top-performing drinks like Latte and Americano with Milk during peak hours to maximize revenue.
- **Upselling opportunities**: Encourage add-ons or combos for mid-level sellers like Capuccino and Cortado to boost average ticket.
- **Inventory planning**: Ensure adequate stock of high-demand drinks, especially during morning and afternoon peaks.

In [12]:
top_sellers = run_query("""
SELECT coffee_name, 
       COUNT(*) AS total_sales, 
       ROUND(SUM(money), 2) AS total_revenue,
       ROUND(AVG(money), 2) AS avg_price
FROM coffee_sales
GROUP BY coffee_name
ORDER BY total_revenue DESC
LIMIT 5;
""", conn)

print(top_sellers.to_string(index=False)) # to_string to avoid truncation

        coffee_name  total_sales  total_revenue  avg_price
              Latte          757       26875.30      35.50
Americano with Milk          809       24751.12      30.59
         Cappuccino          486       17439.14      35.88
          Americano          564       14650.26      25.98
      Hot Chocolate          276        9933.46      35.99


#### 6.2 Peak Hours

- **10 AM is the peak sales hour**, generating $10,198 in revenue. This suggests mornings are the busiest period, likely due to office commuters and the customary "morning coffee" routine.

- **1–2 PM sees a slight dip** in revenue (~$7,100), which could be an opportunity for **promotions or lunch combos** to boost revenue during this quieter period.

- **7 PM–9 PM** ($6,400–$7,700), still generates decent revenue (~$6,398–$7,752), but lower than afternoon. People still buy coffee in the evening; maybe offer **seasonal warm drinks** like Hot Chocolate to increase evening sales.

- **6-7 AM** are low hours: $149.40 (6 AM) and $2,846.02 (7 AM). Very early opening may not be worth staffing heavily unless you have loyal early-morning customers. Could **consider reducing staff or offering pre-order options**.

**Strategy Recommendations:**
- **Staffing**: Allocate more baristas from 9 AM–12 PM and 4 PM–5 PM to handle peak demand.

- **Product promotions**: Target slow hours (1–3 PM) with discounts or combos to increase average revenue.

- **Menu focus**: Highlight high-margin drinks during peak hours to maximize profits.

- **Operational planning**: Monitor inventory for popular drinks during peak hours to avoid shortages.

In [13]:
peak_hours = run_query("""
SELECT hour_of_day, 
       ROUND(SUM(money), 2) AS total_revenue,
       COUNT(*) AS transactions
FROM coffee_sales
GROUP BY hour_of_day
ORDER BY hour_of_day;
""", conn)

print(peak_hours.to_string(index=False))
print(f"\nPeak Hour: {peak_hours.loc[peak_hours['total_revenue'].idxmax(), 'hour_of_day']}:00")
print(f"Peak Revenue: ${peak_hours['total_revenue'].max():,.2f}")

 hour_of_day  total_revenue  transactions
           6         149.40             5
           7        2846.02            88
           8        7017.88           235
           9        7264.28           242
          10       10198.52           328
          11        8453.10           283
          12        7419.62           241
          13        7028.76           225
          14        7173.80           225
          15        7476.02           236
          16        9031.84           278
          17        7659.76           237
          18        7162.60           218
          19        7751.96           229
          20        5578.92           169
          21        6397.94           195
          22        3635.16           113

Peak Hour: 10:00
Peak Revenue: $10,198.52


#### 6.3 Revenue by Day of the Week

- **Tuesday generates the highest revenue** with $18,168.38, suggesting mid-week demand is strongest.

- **Monday** follows closely at $17,363.10, showing strong early-week sales, likely as people start their workweek.

- **Friday** brings in $16,802.66, still significant but slightly lower than earlier in the week, which could reflect early weekend patterns.

- **Thursday ($16,091.40) and Wednesday ($15,750.46)** show steady mid-week revenue.

- **Weekends see lower revenue**: Saturday at $14,733.52 and Sunday at $13,336.06, indicating less foot traffic compared to weekdays.

**Strategy Recommendations:**

- **Staffing**: Schedule more staff on weekdays, particularly Tuesday and Monday, to handle higher demand.

- **Promotions**: Offer weekend promotions to boost sales during slower days.

- **Inventory planning**: Ensure top-selling drinks are well-stocked during high-revenue weekdays.

In [15]:
revenue_by_day = run_query("""
SELECT Weekday, 
       ROUND(SUM(money), 2) AS total_revenue,
       COUNT(*) AS transactions,
       ROUND(AVG(money), 2) AS avg_transaction
FROM coffee_sales
GROUP BY Weekday
ORDER BY Weekdaysort;
""", conn)

print(revenue_by_day.to_string(index=False))

Weekday  total_revenue  transactions  avg_transaction
    Mon       17363.10           544            31.92
    Tue       18168.38           572            31.76
    Wed       15750.46           500            31.50
    Thu       16091.40           510            31.55
    Fri       16802.66           532            31.58
    Sat       14733.52           470            31.35
    Sun       13336.06           419            31.83


#### 6.4 Average Sale per Hour

- **Morning transactions (6–9 AM)** tend to be smaller, averaging around $29–$32 per sale, reflecting lighter orders early in the day.  

- **Late morning to early afternoon (10 AM–2 PM)** sees moderate average sales ($31–$32), coinciding with peak traffic hours.  

- **Afternoon and evening (3–9 PM)** have the highest average transaction values, peaking at $33.85 around 7 PM, suggesting customers are purchasing larger or "premium" drinks later in the day.  

- **Night hours (10–11 PM)** maintain relatively high averages despite fewer customers, indicating that fewer orders are slightly bigger in value.

**Strategy Recommendations:**

- **Upselling:** Promote premium drinks or add-ons in the afternoon and evening to maximize revenue per transaction.  

- **Early morning offers:** Introduce combos or incentives to increase average sales when traffic is lighter. 

In [16]:
avg_sale_hour = run_query("""
SELECT hour_of_day,
       ROUND(AVG(money), 2) AS avg_sale_per_hour,
       COUNT(*) AS transactions
FROM coffee_sales
GROUP BY hour_of_day
ORDER BY hour_of_day;
""", conn)

print(avg_sale_hour.to_string(index=False))

 hour_of_day  avg_sale_per_hour  transactions
           6              29.88             5
           7              32.34            88
           8              29.86           235
           9              30.02           242
          10              31.09           328
          11              29.87           283
          12              30.79           241
          13              31.24           225
          14              31.88           225
          15              31.68           236
          16              32.49           278
          17              32.32           237
          18              32.86           218
          19              33.85           229
          20              33.01           169
          21              32.81           195
          22              32.17           113


#### 6.5 Monthly Sales Performance: Growth Rate by Month

- **Stable start**: Mar–Apr stay steady, with only a slight dip (–3%).

- **Spring lift**: May rises sharply (+43%), the first strong upward shift.

- **Summer softness**: Jun–Jul decline (–7% to –9%), signaling weaker mid-year performance.

- **Late-summer recovery**: Aug rebounds (+10%) after July’s drop.

- **Peak season**: Sep–Oct deliver the strongest growth (+31% and +39%).

- **Sharp correction**: Nov plunges (–38%), the steepest fall in the cycle.

- **Year-end steadiness**: Dec slips only –4%, holding more stable than November.

- **Post-holiday slump**: Jan 2025 drops –22%, typical for early-year slowdowns.

- **Major rebound**: Feb 2025 surges (+107%), doubling revenue.

- **Spring adjustment**: Mar 2025 falls –24%, normalizing after February’s spike.

**Strategy Recommendations**:
- Prioritize staffing and inventory for Sep–Oct, when demand peaks.
- Use May and Feb surges to introduce new drinks or limited-time offers.
- Create targeted promotions or loyalty incentives during weak periods (Jun–Jul, Nov, Jan).



In [18]:
# Use month_sequence for proper ordering
monthly_sales = df.groupby(['year_month_display', 'month_sequence', 'year_month_sort'])['money'].agg([
    ('total_revenue', 'sum'),
    ('transactions', 'count'),
    ('avg_transaction', 'mean')
]).reset_index()
monthly_sales = monthly_sales.sort_values('month_sequence')

# Calculate growth rate
monthly_sales['growth_rate'] = (
    monthly_sales['total_revenue'].pct_change().fillna(0) * 100
).round(2)

# Display with proper labels
monthly_display = monthly_sales[['year_month_display', 'total_revenue', 'transactions', 'growth_rate']].copy()
monthly_display['total_revenue'] = monthly_display['total_revenue'].apply(lambda x: f"${x:,.2f}")
monthly_display['growth_rate'] = monthly_display['growth_rate'].apply(lambda x: f"{x:.2f}%")
monthly_display.columns = ['Period', 'Revenue', 'Transactions', 'Growth Rate']
print(monthly_display.to_string(index=False))

  Period    Revenue  Transactions Growth Rate
Mar 2024  $5,905.20           175       0.00%
Apr 2024  $5,719.56           168      -3.14%
May 2024  $8,164.42           241      42.75%
Jun 2024  $7,617.76           223      -6.70%
Jul 2024  $6,915.94           237      -9.21%
Aug 2024  $7,613.84           272      10.09%
Sep 2024  $9,988.64           344      31.19%
Oct 2024 $13,891.16           426      39.07%
Nov 2024  $8,590.54           259     -38.16%
Dec 2024  $8,237.74           259      -4.11%
Jan 2025  $6,398.86           201     -22.32%
Feb 2025 $13,215.48           423     106.53%
Mar 2025  $9,986.44           319     -24.43%


#### 6.6 Sales by Time of Day

- **Morning**: 1,181 transactions generating $35,929, with an average ticket of $30.42 — steady but slightly lower-value sales.

- **Afternoon**: 1,205 transactions and $38,130 in revenue, driven by a higher average value ($31.64).

- **Night**: 1,161 transactions producing $38,186, the highest average ticket ($32.89), indicating stronger spending later in the day.

**Strategy Recommendations**:

- Promote breakfast combos or loyalty perks in the morning to lift the lower average ticket.

- Capitalize on strong afternoon traffic with targeted upsells (iced drinks, add-ons).

- Leverage high-value night purchases with premium drinks or limited evening specials to maximize spending.

In [20]:
time_of_day_analysis = run_query("""
SELECT Time_of_Day,
       COUNT(*) AS transactions,
       ROUND(SUM(money), 2) AS total_revenue,
       ROUND(AVG(money), 2) AS avg_transaction
FROM coffee_sales
GROUP BY Time_of_Day
ORDER BY 
    CASE Time_of_Day
        WHEN 'Morning' THEN 1
        WHEN 'Afternoon' THEN 2
        WHEN 'Night' THEN 3
    END;
""", conn)

print(time_of_day_analysis.to_string(index=False))

Time_of_Day  transactions  total_revenue  avg_transaction
    Morning          1181       35929.20            30.42
  Afternoon          1205       38130.04            31.64
      Night          1161       38186.34            32.89


### 7. Outliers: Extreme Sales

#### 7.1 Overview

- Transaction count: 1,415 purchases exceed the high-value threshold, representing the top tier of sales.

- Revenue impact: These transactions generate $51,511.80, accounting for 46% of total revenue, highlighting their outsized contribution.

- Average value: Each high-value transaction averages $36.40, slightly above typical sales, confirming that larger purchases are common.

In [22]:
# Threshold for top 25% transactions
threshold = df['money'].quantile(0.75)
high_value_sales = df[df['money'] >= threshold]

# Summary statistics for high-value transactions
num_outliers = len(high_value_sales)
total_outliers = high_value_sales['money'].sum()
avg_outliers = high_value_sales['money'].mean()
pct_of_total = total_outliers / df['money'].sum() * 100

# Display summary
summary = pd.DataFrame({
    "Metric": [
        "High-value transactions", 
        "Total revenue", 
        "Average value", 
        "% of total revenue",
        "Threshold value"
    ], 
    "Value": [
        f"{num_outliers:,}",
        f"${total_outliers:,.2f}", 
        f"${avg_outliers:.2f}", 
        f"{pct_of_total:.2f}%",
        f"${threshold:.2f}"
    ]
})
print(summary.to_string(index=False))

                 Metric      Value
High-value transactions      1,415
          Total revenue $51,511.80
          Average value     $36.40
     % of total revenue     45.89%
        Threshold value     $35.76


#### 7.2 High-Value Coffee Transactions: Item Contribution to Top Sales

- **Top contributor**: Latte dominates high-value sales, generating $20,508 (40% of total), with 563 transactions averaging $36.43 each.


- **Strong performers**: Cappuccino follows with $14,288 (28%), 390 transactions, and a similar average transaction value ($36.64).


- **Mid-tier contributors**: Hot Chocolate brings in $9,080 (18%) across 250 transactions, slightly below the top two in both revenue and count.


- **Smaller but notable**: Cocoa adds $7,635 (15%) from 212 transactions, maintaining consistent average value ($36.01).


**Strategy Recommendations**: 
- Upselling or promotions targeting Lattes and Cappuccinos could maximize revenue, while Hot Chocolate and Cocoa represent steady secondary options for high-value sales.




In [26]:
# Aggregate high-value sales by coffee item
coffee_high_value = high_value_sales.groupby('coffee_name').agg(
    total_revenue=('money', 'sum'), # total revenue from high-value sales
    transaction_count=('money', 'count'), # number of high-value transactions
    avg_value=('money', 'mean') # average value of high-value transactions
).sort_values(by='total_revenue', ascending=False)

# Calculate percentage of high-value revenue for each coffee item
coffee_high_value['pct_of_high_value_revenue'] = (
    coffee_high_value['total_revenue'] / high_value_sales['money'].sum() * 100
).round(2)

# Format monetary values and percentages for display
coffee_high_value['total_revenue'] = coffee_high_value['total_revenue'].apply(lambda x: f"${x:,.2f}")
coffee_high_value['avg_value'] = coffee_high_value['avg_value'].apply(lambda x: f"${x:.2f}")
coffee_high_value['pct_of_high_value_revenue'] = coffee_high_value['pct_of_high_value_revenue'].apply(lambda x: f"{x:.2f}%")

print(coffee_high_value)

              total_revenue  transaction_count avg_value  \
coffee_name                                                
Latte            $20,508.22                563    $36.43   
Cappuccino       $14,288.42                390    $36.64   
Hot Chocolate     $9,080.14                250    $36.32   
Cocoa             $7,635.02                212    $36.01   

              pct_of_high_value_revenue  
coffee_name                              
Latte                            39.81%  
Cappuccino                       27.74%  
Hot Chocolate                    17.63%  
Cocoa                            14.82%  


### 8. Predictive Analysis

#### 8.1 Hourly Sales Forecast (6:00 - 22:00)
Predicting expected revenue for each hour of the day using polynomial regression to capture peak and off-peak trends.

- **Peak sales** expected between 16:00–22:00, with predicted revenue gradually increasing throughout the day.

In [ ]:
# Aggregate average sales by hour
hourly_sales = df.groupby('hour_of_day')['money'].mean().reset_index()
X = hourly_sales[['hour_of_day']]
y = hourly_sales['money']

# Polynomial regression model (degree 2), for capturing peak/off-peak trends
poly_hour_model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly_hour_model.fit(X, y)

# Predict average sales for each hour from 6:00 to 22:00
future_hours = pd.DataFrame({'hour_of_day': range(6, 23)})
predicted_hourly_sales = poly_hour_model.predict(future_hours)

# Format and display predictions
predicted_hourly_sales_df = future_hours.copy()
predicted_hourly_sales_df['predicted_avg_transaction'] = predicted_hourly_sales
predicted_hourly_sales_df['predicted_avg_transaction'] = predicted_hourly_sales_df['predicted_avg_transaction'].apply(
    lambda x: f"${x:.2f}"
)
predicted_hourly_sales_df['hour'] = predicted_hourly_sales_df['hour_of_day'].apply(lambda x: f"{x}:00")
print(predicted_hourly_sales_df[['hour', 'predicted_avg_transaction']].to_string(index=False))

# Note: I used polynomial regression with degree 2 because the hourly sales data showed a curved pattern - sales rise during morning rush, plateau midday, and change throughout the evening. A simple linear model couldn't capture this non-linear relationship, while degree 2 provided the right amount of curvature without overfitting.

 hour predicted_avg_transaction
 6:00                    $30.16
 7:00                    $30.34
 8:00                    $30.52
 9:00                    $30.70
10:00                    $30.89
11:00                    $31.07
12:00                    $31.26
13:00                    $31.45
14:00                    $31.64
15:00                    $31.83
16:00                    $32.02
17:00                    $32.22
18:00                    $32.41
19:00                    $32.61
20:00                    $32.81
21:00                    $33.02
22:00                    $33.22


#### 8.2 Monthly Sales Forecast (Next 3 Months)
Predicting expected monthly revenue based on historical trends using polynomial regression, including simulated next months.

- Revenue is projected to **increase gradually**, reaching ~$11,076 in March.

In [ ]:
# Use month_sequence for proper temporal ordering
monthly_sales_seq = df.groupby('month_sequence')['money'].sum().reset_index()
monthly_sales_seq = monthly_sales_seq.sort_values('month_sequence')

# Polynomial regression model (degree 2) for monthly sales trend, capturing non-linear growth
X_month = monthly_sales_seq[['month_sequence']]
y_month = monthly_sales_seq['money']

poly_month_model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly_month_model.fit(X_month, y_month)

# Predict next 3 months after the last month in your data
max_month = df['month_sequence'].max()
future_months = pd.DataFrame({'month_sequence': [max_month + 1, max_month + 2, max_month + 3]})
predicted_monthly_sales = poly_month_model.predict(future_months)

# Create readable labels based on your last date
last_date = df['Date'].max()
future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=3, freq='MS')

# Format and display predictions
predicted_monthly_sales_df = pd.DataFrame({
    'Period': future_dates.strftime('%b %Y'),
    'Predicted Revenue': [f"${x:,.2f}" for x in predicted_monthly_sales],
    'Month Sequence': [max_month + 1, max_month + 2, max_month + 3]
})
print(predicted_monthly_sales_df.to_string(index=False))

  Period Predicted Revenue  Month Sequence
May 2025        $10,021.35              14
Jun 2025         $9,886.23              15
Jul 2025         $9,684.47              16


### 9. Executive Summary

In [32]:
# Get date range information
first_month = df.loc[df['Date'].idxmin(), 'year_month_display']
last_month = df.loc[df['Date'].idxmax(), 'year_month_display']

# Calculate summary statistics
total_revenue = df['money'].sum()
total_transactions = len(df)
avg_transaction = df['money'].mean()
unique_products = df['coffee_name'].nunique()

# Display summary
print(f"Analysis Period: {first_month} to {last_month}")
print(f"Total Duration: {df['month_sequence'].nunique()} months")
print(f"Date Range: {df['Date'].min().date()} to {df['Date'].max().date()}")
print(f"\nTotal Transactions: {total_transactions:,}")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Average Transaction: ${avg_transaction:.2f}")
print(f"Unique Products: {unique_products}")
print(f"\nPeak Hour: {peak_hours.loc[peak_hours['total_revenue'].idxmax(), 'hour_of_day']}:00 (${peak_hours['total_revenue'].max():,.2f})")
print(f"Best Day: {revenue_by_day.loc[revenue_by_day['total_revenue'].idxmax(), 'Weekday']} (${revenue_by_day['total_revenue'].max():,.2f})")
print(f"Top Product: {top_sellers.iloc[0]['coffee_name']} (${top_sellers.iloc[0]['total_revenue']:,.2f} revenue)")

Analysis Period: Mar 2024 to Mar 2025
Total Duration: 13 months
Date Range: 2024-03-01 to 2025-03-23

Total Transactions: 3,547
Total Revenue: $112,245.58
Average Transaction: $31.65
Unique Products: 8

Peak Hour: 10:00 ($10,198.52)
Best Day: Tue ($18,168.38)
Top Product: Latte ($26,875.30 revenue)


### 10. Close the Connection

In [ ]:
conn.close()